In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from ema_workbench import load_results
from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator, 
                           ema_logging, Constant, Scenario, Policy, CategoricalParameter, IntegerParameter,
                           perform_experiments, SequentialEvaluator, Constraint)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
from ema_workbench.util import ema_logging, utilities
import time
from problem_formulation import get_model_for_problem_formulation
#from signal import signal, SIGPIPE, SIG_DFL
#signal(SIGPIPE, SIG_DFL)

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [6]:
results=pd.read_csv("results/optimization_test_pols.csv")
results.drop(["Unnamed: 0"], inplace=True,axis=1)
results

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,All Costs,Expected Number of Deaths,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths
0,1,0,0,0,0,0,1,0,1,0,...,6,6,6,3,3,9.080365e+08,0.0,0.0,2.097396e+08,0.0
1,0,1,0,0,0,0,0,1,1,0,...,3,3,10,2,7,1.179664e+09,0.0,0.0,1.598583e+08,0.0
2,1,1,0,0,1,0,1,0,0,1,...,5,4,4,3,1,1.523291e+09,0.0,0.0,4.784795e+07,0.0


In [7]:
#Filter out candidate solutions based on logical statements
#outcome_df=results.iloc[:,-5:]
#outcome_df

,All Costs,Expected Number of Deaths,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths
0,9.080365e+08,0.0,0.0,2.097396e+08,0.0
1,1.179664e+09,0.0,0.0,1.598583e+08,0.0
2,1.523291e+09,0.0,0.0,4.784795e+07,0.0


In [10]:
log_all_costs = results["All Costs"] < results["All Costs"].quantile(0.5)
log_a1_deaths = results["A.1_Expected Number of Deaths"] < results["A.1_Expected Number of Deaths"].quantile(0.7)
log_a1_dike = results["A.1_Dike Investment Costs"] < results["A.1_Dike Investment Costs"].quantile(0.3)
log_a1_dam = results["A.1_Expected Annual Damage"] < results["A.1_Expected Annual Damage"].quantile(0.5)
log_all_deaths = results["Expected Number of Deaths"] < results["Expected Number of Deaths"].quantile(0.57)

In [13]:
policies = results[log_all_costs]
policies = policies[log_a1_deaths]
policies = policies[log_a1_dike]
policies = policies[log_a1_dam]
policies = policies[log_all_deaths]

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
policies = policies.drop([o.name for o in model.outcomes], axis=1)
policies

In [ ]:
from ema_workbench import Policy

policies_to_evaluate = []

for i, policy in policies.iterrows():
    policies_to_evaluate.append(Policy(str(i), **policy.to_dict()))

In [ ]:
n_scenarios = 1000
with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies_to_evaluate)